### Modelo de aprendizado de máquina para reconhecimento de imagens de frutas ou verduras

In [10]:
import sklearn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall

In [11]:
# Coeficiente de aleatoriedade
tf.random.set_seed(4)

In [12]:
# Caminho para dados de treino
caminho_dados_treino = Path('fruits-360/Training')
# Caminho para dados de teste
caminho_dados_teste = Path('fruits-360/Test')

In [13]:
# Listagem do conteúdo da pasta
imagens_treino = list(caminho_dados_treino.glob('*/*'))
# Extração de apenas o valor com o caminho de cada imagem (removendo PosixPath)
imagens_treino = list(map(lambda x: str(x), imagens_treino))
imagens_treino = list(map(lambda x: str(x).replace('\\', '/'), imagens_treino))

In [14]:
imagens_treino[925:936]
# O nome dos arquivos prejudica a compreensão

['fruits-360/Training/Apple Crimson Snow/r_90_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_91_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_92_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_93_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_94_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_95_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_96_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_97_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_98_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_99_100.jpg',
 'fruits-360/Training/Apple Crimson Snow/r_9_100.jpg']

##### Pré-processamento de dados

In [15]:
# Obtenção do label de cada imagem
def extrai_label(caminho_imagem):
    return caminho_imagem.split("/")[-2]

imagens_treino_labels = list(map(lambda x: extrai_label(x), imagens_treino))

In [16]:
imagens_treino_labels[840:844]
# O sistema não reconhece strings, para isso, é necessário converte-la em números

['Apple Crimson Snow',
 'Apple Crimson Snow',
 'Apple Crimson Snow',
 'Apple Crimson Snow']

In [17]:
# Criação do objeto
encoder = LabelEncoder()

imagens_treino_labels = encoder.fit_transform(imagens_treino_labels)

# Aplicação do one-hot-encoding nos labels
imagens_treino_labels = tf.keras.utils.to_categorical(imagens_treino_labels)

In [18]:
# Divisão dos dados de treino e de validação
x_treino, x_valid, y_treino, y_valid = train_test_split(imagens_treino, imagens_treino_labels)

##### Dataset Augmentation

Consiste em modificar as imagens para que o modelo fique mais preciso em identificar

In [20]:
# Redimensionamento das imagens
img_size = 224
resize = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Resizing(img_size, img_size)])

# Criação do objeto do Data Augmentation
data_augmentation = tf.keras.Sequential([RandomFlip("horizontal"),
                                         RandomRotation(0.2),
                                         RandomZoom(height_factor = (-0.3,-0.2)) ])


##### Preparo dos dados

In [25]:
# Hiperparametros
batch_size = 32
autotune = tf.data.experimental.AUTOTUNE


In [23]:
# Carregamento e transformação de imagens
def carrega_transforma(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    return image, label

In [21]:
# Preparação dos dados no formato do TensorFlow
# Dataset augmentation é usado apenas em treino
def prepara_dataset(path, labels, train = True):
    image_paths = tf.convert_to_tensor(path)
    labels = tf.convert_to_tensor(labels)
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    dataset = dataset.map(lambda image, label: carrega_transforma(image, label)) 
    dataset = dataset.map(lambda image, label: (resize(image), label), num_parallel_calls = autotune)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)

    # Se train = True aplica dataset augmentation
    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label), num_parallel_calls = autotune)
  
    # Se train = False repete sobre o dataset e retorna
    dataset = dataset.repeat()

    return dataset


In [26]:
# Criação do dataset de treino
dataset_treino = prepara_dataset(x_treino, y_treino)

In [ ]:
# Shape
imagem, label = next(iter(dataset_treino))
print(imagem.shape) # "Lotes" de 32 imagens 224 x 244 de tamanho e 3 canais de cores (RGB)
print(label.shape) # 32 labels e um vetor de 131 itens na pasta

In [ ]:
# Criação do dataset de validação
dataset_valid = prepara_dataset(X_valid, y_valid, train = False)

##### Construção do modelo

In [35]:
# Carregamento de um modelo pre-treinado
modelo_pre = EfficientNetB3(input_shape = (224, 224, 3), include_top = False)

In [36]:
# Adição das camadas ao modelo_pre
modelo = tf.keras.Sequential([modelo_pre,
                            tf.keras.layers.GlobalAveragePooling2D(),
                            tf.keras.layers.Dense(131, activation = 'softmax')])

In [39]:
# Sumario do modelo
modelo.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 7, 7, 1536)       10783535  
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 131)               201347    
                                                                 
Total params: 10,984,882
Trainable params: 10,897,579
Non-trainable params: 87,303
_________________________________________________________________


In [38]:
# Hiperparâmetros
lr = 0.001
beta1 = 0.9
beta2 = 0.999
ep = 1e-07

In [ ]:
# Compilação do modelo
modelo.compile(optimizer = Adam(learning_rate = lr, 
                                beta_1 = beta1, 
                                beta_2 = beta2, 
                                epsilon = ep),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy', Precision(name = 'precision'), Recall(name = 'recall')])
